<a href="https://colab.research.google.com/github/mahmoodtt1/OR/blob/main/Signal_Coordination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from pyomo.environ import *

# install needed libraries

!pip install networkx matplotlib

# install pyomo solvers.

!apt-get install -y glpk-utils

!apt-get install -y coinor-cbc


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0-1 [361 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 glpk-ut

In [8]:
# Create model
model = ConcreteModel()

# Sets
model.I = Set(initialize=['1'])  # One phase, can be extended

# Parameters
C = 60                          # Cycle time (seconds)
t_AB = 15                       # Travel time from A to B
g_min = 5
g_max = 30
M = 100                         # Large constant

model.C = Param(initialize=C)
model.t_AB = Param(initialize=t_AB)
model.g_min = Param(initialize=g_min)
model.g_max = Param(initialize=g_max)
model.M = Param(initialize=M)

# Variables
model.s_A = Var(model.I, bounds=(0, C))
model.s_B = Var(model.I, bounds=(0, C))
model.g_A = Var(model.I, bounds=(g_min, g_max))
model.g_B = Var(model.I, bounds=(g_min, g_max))
model.e_A = Var(model.I)
model.e_B = Var(model.I)

model.o = Var(model.I, within=NonNegativeReals)  # Overlap
model.early_end = Var(model.I)
model.late_start = Var(model.I)

model.x = Var(model.I, within=Binary)  # For min()
model.y = Var(model.I, within=Binary)  # For max()

# Constraints

def end_A_rule(m, i):
    return m.e_A[i] == m.s_A[i] + m.g_A[i]
model.end_A = Constraint(model.I, rule=end_A_rule)

def end_B_rule(m, i):
    return m.e_B[i] == m.s_B[i] + m.g_B[i]
model.end_B = Constraint(model.I, rule=end_B_rule)

# Min of (e_A, e_B - t_AB)
def min1_upper1(m, i):
    return m.early_end[i] <= m.e_A[i]
def min1_upper2(m, i):
    return m.early_end[i] <= m.e_B[i] - t_AB
def min1_lower1(m, i):
    return m.early_end[i] >= m.e_A[i] - m.M * (1 - m.x[i])
def min1_lower2(m, i):
    return m.early_end[i] >= m.e_B[i] - t_AB - m.M * m.x[i]

model.min1_u1 = Constraint(model.I, rule=min1_upper1)
model.min1_u2 = Constraint(model.I, rule=min1_upper2)
model.min1_l1 = Constraint(model.I, rule=min1_lower1)
model.min1_l2 = Constraint(model.I, rule=min1_lower2)

# Max of (s_A, s_B - t_AB)
def max1_lower1(m, i):
    return m.late_start[i] >= m.s_A[i]
def max1_lower2(m, i):
    return m.late_start[i] >= m.s_B[i] - t_AB
def max1_upper1(m, i):
    return m.late_start[i] <= m.s_A[i] + m.M * (1 - m.y[i])
def max1_upper2(m, i):
    return m.late_start[i] <= m.s_B[i] - t_AB + m.M * m.y[i]

model.max1_l1 = Constraint(model.I, rule=max1_lower1)
model.max1_l2 = Constraint(model.I, rule=max1_lower2)
model.max1_u1 = Constraint(model.I, rule=max1_upper1)
model.max1_u2 = Constraint(model.I, rule=max1_upper2)

# Overlap definition
def overlap_def(m, i):
    return m.o[i] == m.early_end[i] - m.late_start[i]
model.overlap_def = Constraint(model.I, rule=overlap_def)

# Ensure non-negative overlap
def overlap_nonneg(m, i):
    return m.o[i] >= 0
model.overlap_pos = Constraint(model.I, rule=overlap_nonneg)

# Objective: Maximize total overlap
def objective_rule(m):
    return sum(m.o[i] for i in m.I)
model.obj = Objective(rule=objective_rule, sense=maximize)


In [9]:
solver = SolverFactory("glpk", executable="/usr/bin/glpsol")

results = solver.solve(model, tee=True)


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpz8xt2va6.glpk.raw --wglp /tmp/tmp0tb9ddf_.glpk.glp --cpxlp
 /tmp/tmpz045mx5d.pyomo.lp
Reading problem data from '/tmp/tmpz045mx5d.pyomo.lp'...
/tmp/tmpz045mx5d.pyomo.lp:88: warning: lower bound of variable 'x15' redefined
/tmp/tmpz045mx5d.pyomo.lp:88: warning: upper bound of variable 'x15' redefined
12 rows, 11 columns, 30 non-zeros
2 integer variables, all of which are binary
90 lines were read
Writing problem data to '/tmp/tmp0tb9ddf_.glpk.glp'...
75 lines were written
GLPK Integer Optimizer 5.0
12 rows, 11 columns, 30 non-zeros
2 integer variables, all of which are binary
Preprocessing...
3 constraint coefficient(s) were reduced
11 rows, 8 columns, 26 non-zeros
2 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+02  ratio =  1.000e+02
GM: min|aij| =  9.554e-01  max|aij| =  1.047e+00  ratio =  1.095e+00
EQ: min|aij| =  9.129e-01  max|aij| =  1.

In [10]:
for i in model.I:
    print(f"Phase {i}:")
    print(f"  Start A: {value(model.s_A[i])}")
    print(f"  Green A: {value(model.g_A[i])}")
    print(f"  Start B: {value(model.s_B[i])}")
    print(f"  Green B: {value(model.g_B[i])}")
    print(f"  Overlap: {value(model.o[i])}")


Phase 1:
  Start A: 0.0
  Green A: 30.0
  Start B: 15.0
  Green B: 30.0
  Overlap: 30.0
